In [16]:

pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
  Using cached torchvision-0.24.0-cp313-cp313-win_amd64.whl.metadata (5.9 kB)
Using cached torchvision-0.24.0-cp313-cp313-win_amd64.whl (4.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets  
from torchvision import transforms       
from torch.utils.data import DataLoader  

In [ ]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_data=datasets.CIFAR100(root='./data',train=True,download=True,transform=transform)
test_data=datasets.CIFAR100(root='./data',train=False,download=True,transform=transform)

In [ ]:
train_loader=DataLoader(train_data,batch_size=128,shuffle=True)
test_loader=DataLoader(test_data,batch_size=128,shuffle=False)# test need not to be shuffled
#batch--row doesn't affect the neuron network performance, it just affects the training speed and memory usage.
# In general larger batch sizes will use more memory, and smaller batch sizes will train slower.


In [22]:
#architecture

class CIFAR100MLP(nn.Module):
    def __init__(self):
        super(CIFAR100MLP, self).__init__()
        #no of layers = 5
        #layer_name=nn.linear(i/p neurons,o/p neurons)  
        self.flatten = nn.Flatten() #flatten layer it is doing what means is converting 2D image to 1D array
        self.fc1 = nn.Linear(32*32*3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 100)  # 100 classes for CIFAR-100
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

In [32]:
model=CIFAR100MLP()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)#lr=learning rate

In [37]:
epoch_loss=0.0
for epoch  in range(10):
    for image,label in train_loader:
        #send the image to model
        output=model(image)
        #loss calculation
        loss=criterion(output,label)
        #backpropagation
        loss.backward()
        #optimizer step
        optimizer.zero_grad()
        epoch_loss+=loss.item()
    print(f"epoch_loss: {epoch_loss}")#loss is scalar value so we use item() to get the value





epoch_loss: 1801.0055270195007
epoch_loss: 3602.0123896598816
epoch_loss: 5403.018153190613
epoch_loss: 7204.017608642578
epoch_loss: 9005.023203849792
epoch_loss: 10806.02914094925
epoch_loss: 12607.03457069397
epoch_loss: 14408.041017055511
epoch_loss: 16209.046726226807
epoch_loss: 18010.052703857422


In [38]:
model.eval()
total=0
correct=0
with torch.no_grad():
    for image,label in test_loader:
        output=model(image)
        _,predicted=torch.max(output,1)#.max returns two values max value(neglected) and index of max value
        total+=label.size(0)     
        correct+=(predicted==label).sum().item() 
    print('Accuracy: {:.2f}%'.format(100*correct/total))    

Accuracy: 0.92%
